<a href="https://colab.research.google.com/github/Ryouko-Yamanda65777/stablecolab/blob/main/stablecolab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# stablecolab


**_a project to run SD model with colab_**

In [ ]:

#@title Install dependencies



!pip install stablepy==0.5.1



# download model


%cd /content/


# VAE
!wget https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/anything_fp16.safetensors


# Embeddings
!wget https://huggingface.co/embed/negative/resolve/main/bad-hands-5.pt
!wget https://huggingface.co/embed/negative/resolve/main/bad-artist.pt

# Upscaler
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth



!wget https://huggingface.co/Ryouko65777/RealESRGAN_x4plus_anime_6B/resolve/main/RealESRGAN_x4plus_anime_6B.pth -O /content/upscaler/RealESRGAN_x4plus_anime_6B.pth

In [ ]:
#@title downalod lora model
import os
import time
import urllib.request
from pathlib import Path
from urllib.parse import urlparse, parse_qs, unquote

# Set variables manually since we can't use argparse in Colab
url = 'https://civitai.com/api/download/models/834507' #@param {type:"string"}
  # Replace with your desired URL
output_path = "/content" #@param {type:"string"}
  # Output directory on Colab

CHUNK_SIZE = 1638400
TOKEN_FILE = Path.home() / '.civitai' / 'config'
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'

def get_token():
    try:
        with open(TOKEN_FILE, 'r') as file:
            token = file.read()
            return token
    except Exception as e:
        return None

def store_token(token: str):
    TOKEN_FILE.parent.mkdir(parents=True, exist_ok=True)
    with open(TOKEN_FILE, 'w') as file:
        file.write(token)

def prompt_for_civitai_token():
    token = "617f17f3e370ff1b3259db73d53d331d"  # Add your actual token here or use an input() function
    store_token(token)
    return token

def download_file(url: str, output_path: str, token: str):
    headers = {
        'Authorization': f'Bearer {token}',
        'User-Agent': USER_AGENT,
    }

    class NoRedirection(urllib.request.HTTPErrorProcessor):
        def http_response(self, request, response):
            return response
        https_response = http_response

    request = urllib.request.Request(url, headers=headers)
    opener = urllib.request.build_opener(NoRedirection)
    response = opener.open(request)

    if response.status in [301, 302, 303, 307, 308]:
        redirect_url = response.getheader('Location')
        parsed_url = urlparse(redirect_url)
        query_params = parse_qs(parsed_url.query)
        content_disposition = query_params.get('response-content-disposition', [None])[0]

        if content_disposition:
            filename = unquote(content_disposition.split('filename=')[1].strip('"'))
        else:
            raise Exception('Unable to determine filename')

        response = urllib.request.urlopen(redirect_url)
    elif response.status == 404:
        raise Exception('File not found')
    else:
        raise Exception('No redirect found, something went wrong')

    total_size = response.getheader('Content-Length')

    if total_size is not None:
        total_size = int(total_size)

    output_file = os.path.join(output_path, filename)

    with open(output_file, 'wb') as f:
        downloaded = 0
        start_time = time.time()

        while True:
            buffer = response.read(CHUNK_SIZE)
            if not buffer:
                break
            downloaded += len(buffer)
            f.write(buffer)

            if total_size is not None:
                progress = downloaded / total_size
                print(f'\rDownloading: {filename} [{progress*100:.2f}%]', end='')

    end_time = time.time()
    print(f'\nDownload completed. File saved as: {filename}')
    print(f'Downloaded in {end_time - start_time:.2f} seconds')

# Run the download
token = get_token()
if not token:
    token = prompt_for_civitai_token()

try:
    download_file(url, output_path, token)
except Exception as e:
    print(f'ERROR: {e}')

recomended use model form [Jhon6666](https://huggingface.co/John6666)

In [ ]:
#@title run sdxl



from stablepy import Model_Diffusers



lora1_path = "./Soft_Boyfriend_Friday_Night_Funkin_Soft__PonyXL.safetensors"  #@param {type:"string"}

prompt = "softbf_fnfsoft, 1girl, tan skin, short hair, aqua hair, pointy ears, 4 fingers, solid oval eyes, chibi" #@param {type:"string"}

base_model_id = "John6666/mala-anime-mix-nsfw-pony-xl-v3-sdxl"  #@param {type:"string"}



# Generate a text-to-image diffusion
model = Model_Diffusers(
    base_model_id,
    task_name= 'txt2img',
)



image, info_image = model(
    prompt,
    num_steps = 30,
    lora_A = lora1_path,
    guidance_scale = 7.5,
    sampler = "DPM++ 2M",
    img_width = 512,
    img_height = 1024,
    upscaler_model_path = "./upscaler/RealESRGAN_x4plus_anime_6B.pth",
    upscaler_increases_size = 1.5,
    hires_steps = 25,
)

image[0]